In [103]:
from api_key import API_KEY
import pandas as pd
import requests as re
import json

headers = {"accept": "application/json"}

api_results = []

In [104]:
def create_api_call(call_type, api_args):
    """A function to help create API Calls.

    This returns the link and headers

    Args:
        call_type - Str: A str that can help select which type of API call to build.
        api_args - Dict: A Dict of arguments to be added to the end of the API call link.
    """

    if call_type == "location_search":
        base_url = "https://api.content.tripadvisor.com/api/v1/location/search?"
        args_url = ""

        # The API Key is added first to easily format the "&" in the url. It is also always required.
        base_url += f"key={api_args['key']}".replace(" ", "%20")

        for arg_name, arg in api_args.items():
            if arg_name == "key":
                continue
            else:
                args_url += f"&{arg_name}={arg}"
    else:
        return "implement more API searches"

    return base_url + args_url

In [105]:
args = {
    "key": API_KEY,
    "searchQuery": "San Francisco",
    "category": "attractions",
    "language": "en",
    "radius": "0.25",
    "radiusUnit": "mi",
}

url = create_api_call("location_search", args)

response = re.get(url, headers=headers)

api_results.append(response)

# Location Search Options

The Trip Advisor API for location search returns 10 locations per search. Specify things like "Hotels", "Attrations", or other search parameters in the searchQuery arg. To get a larger list of locations, a strategy is required, and multiple querries will need to be executed.

Ideas: 
1. Find the 10 best attractions in a city. For each attraction, find the 10 best attractions within 1 mile, 10 miles, and 100 yards. Keep finding attractions until you get a certain amount or hit a query count limiter. 

How to store the response: 
 - Use json loads on the text. 
 - Store in a JSON file. 


# Lat Long Search

San Francisco Spans roughly from (37.80453, -122.51981) to (37.71159, -122.38256)

In [112]:
lat_sep = 37.80453 - 37.31159
long_sep = -122.51981 + 122.38256
print(f"Distance Span: Lat{lat_sep}, long {long_sep}")

Distance Span: Lat0.49293999999999727, long -0.13725000000000875


In [113]:
print("Ten Lat Steps: ", lat_sep / 10)
print("Ten Long Steps: ", long_sep / 10)

Ten Lat Steps:  0.04929399999999973
Ten Long Steps:  -0.013725000000000875


In [145]:
lat_steps = [37.80453 - x * 0.0493949 for x in range(10)]
print(lat_steps)
long_steps = [-122.51981 + x * 0.013725 for x in range(10)]
print(long_steps)

[37.80453, 37.7551351, 37.7057402, 37.6563453, 37.6069504, 37.5575555, 37.5081606, 37.4587657, 37.4093708, 37.3599759]
[-122.51981, -122.50608500000001, -122.49236, -122.47863500000001, -122.46491, -122.45118500000001, -122.43746, -122.42373500000001, -122.41001, -122.396285]


In [146]:
api_out = []

for lat in lat_steps:
    for long in long_steps:
        new_args = args.copy()
        new_args["latLong"] = f"{lat},{long}"

        url = create_api_call("location_search", args)
        response = re.get(url, headers=headers)
        api_out.append(response)

In [147]:
len(api_out)

100

In [148]:
loaded_json = [json.loads(x.text) for x in api_out]

In [149]:
end_data = []
for response_lists in loaded_json:
    for loc_dict in response_lists["data"]:
        end_data.append(loc_dict)

In [150]:
end_data

[{'location_id': '60713',
  'name': 'San Francisco',
  'address_obj': {'state': 'California',
   'country': 'United States',
   'address_string': 'San Francisco, CA'}},
 {'location_id': '81103',
  'name': 'Hyatt Regency San Francisco',
  'address_obj': {'street1': '5 Embarcadero Center',
   'street2': '',
   'city': 'San Francisco',
   'state': 'California',
   'country': 'United States',
   'postalcode': '94111-4800',
   'address_string': '5 Embarcadero Center, San Francisco, CA 94111-4800'}},
 {'location_id': '81397',
  'name': 'Fairmont San Francisco',
  'address_obj': {'street1': '950 Mason St',
   'street2': '',
   'city': 'San Francisco',
   'state': 'California',
   'country': 'United States',
   'postalcode': '94108-6000',
   'address_string': '950 Mason St, San Francisco, CA 94108-6000'}},
 {'location_id': '81222',
  'name': 'Hotel Zephyr San Francisco',
  'address_obj': {'street1': '250 Beach Street',
   'street2': 'Fisherman’s Wharf',
   'city': 'San Francisco',
   'state': 

In [151]:
df = pd.DataFrame.from_records(end_data)

In [152]:
df

,location_id,name,address_obj
0,60713,San Francisco,"{'state': 'California', 'country': 'United Sta..."
1,81103,Hyatt Regency San Francisco,"{'street1': '5 Embarcadero Center', 'street2':..."
2,81397,Fairmont San Francisco,"{'street1': '950 Mason St', 'street2': '', 'ci..."
3,81222,Hotel Zephyr San Francisco,"{'street1': '250 Beach Street', 'street2': 'Fi..."
4,676408,InterContinental San Francisco,"{'street1': '888 Howard Street', 'street2': ''..."
...,...,...,...
995,80747,Stanford Court San Francisco,"{'street1': '905 California Street', 'street2'..."
996,80793,Hotel Nikko San Francisco,"{'street1': '222 Mason St', 'street2': '', 'ci..."
997,81315,Marriott Marquis San Francisco,"{'street1': '780 Mission Street', 'street2': '..."
998,224948,Omni San Francisco Hotel,"{'street1': '500 California St', 'street2': ''..."


In [153]:
df.to_csv("test_out.csv")

In [154]:
first_iteration = set()
end_index = []

for row in df.iterrows():
    if row[1][0] in first_iteration:
        continue
    else:
        first_iteration.add(row[1][0])
        end_index.append(row[0])

In [155]:
filtered_df = df.filter(items=end_index, axis=0)

In [156]:
len(filtered_df)

10

In [157]:
len(df)

1000

In [158]:
df["location_id"].unique()

array(['60713', '81103', '81397', '81222', '676408', '80747', '80793',
       '81315', '224948', '112307'], dtype=object)